In [2]:
# TODO which one?
#git clone https://github.com/lucidrains/iTransformer.git
#import iTransformer
import sys
sys.path.append('/vol/fob-vol7/nebenf21/reinbene/bene/MA/iTransformer') 
from iTransformer import iTransformer
import torch
from pathlib import Path

import torch
import torch.optim as optim
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

from utils import data_handling, helpers, training_functions
import config
import pandas as pd
import random

window_size = 96
pred_length = (96)

print("Import succesfull")

Import succesfull


# Zero shot evaluation

In the following we train and evaluate iTransformer in a 5-fold cross-validation zero shot setting on a 96 timestep input and foecasting horizon.
Each dataset is split into a ratio of 80% for training and 20% for zero-shot evaluation. This is done 5 times. The mse loss of each run is saved for all three datasets.

In [3]:
# the following functions are needed for cross-validation splitting, selecting the correct data chunks
# and doing the training and zero-shot prediction

def cross_validaiton_split(data_dict):
    # define split point
    num_ids = data_dict["train"].size(1)
    columns_per_fold = num_ids // 5

    # define dicts for pre-train/fine-tune splits
    data_dict_source = {}
    data_dict_target = {}

    for key, value in data_dict.items():
        # select 1/5 of ids with each full series
        folds = [value[:, i * columns_per_fold : (i + 1) * columns_per_fold] for i in range(5)]


        # add remainder iteratively to folds if present
        remainder = num_ids % 5
        if remainder:
            for i in range(remainder):
                folds[i] = torch.cat((folds[i], value[:, num_ids - remainder + i:num_ids - remainder + i + 1]), axis=1)

        # define pre-training folds
        source = []
        source.append(torch.cat(folds[1:], dim=1))
        source.append(torch.cat((folds[0], folds[2], folds[3], folds[4]), dim=1))
        source.append(torch.cat((folds[0], folds[1], folds[3], folds[4]), dim=1))
        source.append(torch.cat((folds[0], folds[1], folds[3], folds[4]), dim=1))
        source.append(torch.cat(folds[:4], dim=1))

        data_dict_source[key] = source
        data_dict_target[key] = folds

    return data_dict_source, data_dict_target

def select_cross_val_split(dataset_dict, num_split=0):

    temp_dict = dataset_dict.copy()
    for key, value in temp_dict.items():
       # print(value)
        temp_dict[key] = value[num_split]

    return data_handling.convert_data(temp_dict, window_size, pred_length)


def cross_val_train(dataloader_train, dataloader_val):


    inputs, _ = next(iter(dataloader_train))
    print(inputs.size(2))
    num_variates = inputs.size(2)
    
    # define parameters and create config 
    best_parameters = {'depth': 2, 'dim': 256, 'dim_head': 56, 'heads': 4, 'attn_dropout': 0.2, 'ff_mult': 4, 'ff_dropout': 0.1, 
                    'num_mem_tokens': 4, 'learning_rate': 0.0005}


    model_config = {
        'num_variates': num_variates,
        'lookback_len': window_size,
        'depth': best_parameters["depth"],
        'dim': best_parameters["dim"],
        'num_tokens_per_variate': 1,
        'pred_length': pred_length,
        'dim_head': best_parameters["dim_head"],
        'heads': best_parameters["heads"],
        'attn_dropout': best_parameters["attn_dropout"],
        'ff_mult': best_parameters["ff_mult"],
        'ff_dropout': best_parameters["ff_dropout"],
        'num_mem_tokens': best_parameters["num_mem_tokens"],
        'use_reversible_instance_norm': True,
        'reversible_instance_norm_affine': True,
        'flash_attn': True
    }

    # select available deviec
    device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

    print(f"Using device: {device}")

    # defining all needed instances
    model = iTransformer(**model_config).to(device)
    optimizer = optim.Adam(model.parameters(), lr=best_parameters["learning_rate"])
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    writer = SummaryWriter(log_dir=config.CONFIG_LOGS_PATH["iTransformer_baseline"])

    # run model training as mentioned in the original paper
    epoch = 1

    for epoch in range(1, epoch + 1):
        training_functions.train_one_epoch(epoch, model, device, dataloader_train, dataloader_val, optimizer, scheduler, writer)
    return model.state_dict()


def zero_shot_prediction(test, state_dict):
    best_parameters = {'depth': 2, 'dim': 256, 'dim_head': 56, 'heads': 4, 'attn_dropout': 0.2, 'ff_mult': 4, 'ff_dropout': 0.1, 
                    'num_mem_tokens': 4, 'learning_rate': 0.0005}

    inputs, _ = next(iter(test))
    num_variates = inputs.size(2)

    model_config = {
        'num_variates': num_variates,
        'lookback_len': window_size,
        'depth': best_parameters["depth"],
        'dim': best_parameters["dim"],
        'num_tokens_per_variate': 1,
        'pred_length': pred_length,
        'dim_head': best_parameters["dim_head"],
        'heads': best_parameters["heads"],
        'attn_dropout': best_parameters["attn_dropout"],
        'ff_mult': best_parameters["ff_mult"],
        'ff_dropout': best_parameters["ff_dropout"],
        'num_mem_tokens': best_parameters["num_mem_tokens"],
        'use_reversible_instance_norm': True,
        'reversible_instance_norm_affine': True,
        'flash_attn': True
    }

    device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

    # values are all set to zero (beta) and one (gamma), array needs to be adapted to num_variates
    # learned affine parameters are series specific and need to be relearned for new series
    # value are kept at 1 and 0 for stationary normalization
    state_dict["reversible_instance_norm.beta"] = torch.zeros(num_variates, 1, dtype=torch.float)
    state_dict["reversible_instance_norm.gamma"] = torch.ones(num_variates, 1, dtype=torch.float)

    model = iTransformer(**model_config).to(device)
    model.load_state_dict(state_dict)

    return model, device

In [4]:
# ETL electricity dataset
electricity = data_handling.load_electricity()

# eu electricity dataset
data_tensor = data_handling.eu_electricity_to_tensor()
eu_electricity, _ =  data_handling.train_test_split_eu_elec(data_tensor, standardize=True)

# bavaria dataset
data_tensor = data_handling.load_bavaria_electricity()
bavaria_electricity, _ = data_handling.train_test_split_eu_elec(data_tensor, standardize=True)

datasets = {"electricity" : electricity,
			"europe" : eu_electricity,
			"bavaria" : bavaria_electricity
            }

In [16]:
metrics_df = pd.DataFrame()

for key, values in datasets.items():

    # load dataset and create 5 folds 80/20
    data_dict = values
    data_dict_source, data_dict_target = cross_validaiton_split(data_dict)

    # for saving metrics
    metrics_list = []

    # train on 4 splits and predict on the 5th
    for num_split in range(5):
        # select source split
        train_source, _, val_source = select_cross_val_split(data_dict_source, num_split)
        
        # train model on split
        state_dict = cross_val_train(train_source, val_source)

        # select target split
        _, test_target, _ = select_cross_val_split(data_dict_target, num_split)

        # reshape model
        model, device = zero_shot_prediction(test_target, state_dict)

        # evaluate
        metrics = helpers.full_eval(model, test_target, device)
        metrics_list.append(metrics[96]["mse"].item())

    metrics_df[key] = metrics_list
    
metrics_df.index.name = "Split for zero-shot"
metrics_df.to_csv(config.CONFIG_OUTPUT_PATH["iTransformer_baseline"] / "zero_shot_split_dataset.csv")

Feature batch shape: torch.Size([32, 96, 278])
278
Using device: cuda:1


Epoch: 1:   4%|▍         | 6/151 [00:00<00:06, 22.15it/s]

Epoch: Validating: 100%|██████████| 86/86 [00:01<00:00, 83.09it/s]


Validation metrics: {96: {'mse': tensor(0.2137, device='cuda:1')}, 192: {'mse': 0}, 336: {'mse': 0}, 720: {'mse': 0}}
Feature batch shape: torch.Size([32, 96, 70])
Using device: cuda:1


Epoch: Validating: 100%|██████████| 21/21 [00:00<00:00, 177.35it/s]

...

Epoch: Validating: 100%|██████████| 33/33 [00:00<00:00, 173.66it/s]


Feature batch shape: torch.Size([32, 96, 54])
54
Using device: cuda:1


Epoch: Validating: 100%|██████████| 71/71 [00:00<00:00, 235.19it/s]


Validation metrics: {96: {'mse': tensor(0.0007, device='cuda:1')}, 192: {'mse': 0}, 336: {'mse': 0}, 720: {'mse': 0}}
Feature batch shape: torch.Size([32, 96, 13])
Using device: cuda:1


Epoch: Validating: 100%|██████████| 33/33 [00:00<00:00, 173.16it/s]


# Fine-tune pretrained model on small subset of training dataset and predict on the same test dataset

In [21]:
# now we train on a small subset of the training data for multiple epochs and evaluate again

epoch = 1

for epoch in range(1, epoch + 1):
    training_functions.train_one_epoch(epoch, model, device, dataloader_train, dataloader_validation, optimizer, scheduler, writer)

metrics = helpers.full_eval(model, dataloader_test, device)
metrics

for eval_metric, value in metrics[96].items():
    metrics[96][eval_metric] = value.item()

metrics_df = pd.DataFrame.from_dict(metrics[96], orient='index')
metrics_df.rename(columns={0: key}, inplace=True)

#metrics_df.to_csv(f"{config.CONFIG_OUTPUT_PATH['iTransformer_baseline']}/metrics_{key}_epochs{epoch}_revin.csv")

metrics

Epoch: 1: 100%|██████████| 948/948 [00:27<00:00, 34.14it/s]


Epoch 1, MSE-Loss: 0.09739486943883231, LR: 0.0005


Epoch: Validating: 100%|██████████| 131/131 [00:01<00:00, 103.51it/s]


Validation metrics: {96: {'mse': tensor(0.6840, device='cuda:1')}, 192: {'mse': 0}, 336: {'mse': 0}, 720: {'mse': 0}}


Epoch: Validating: 100%|██████████| 267/267 [00:03<00:00, 86.97it/s]


{96: {'mse': 1.006528615951538,
  'mae': 0.5427948236465454,
  'p10': 0.0423695333302021,
  'p50': 0.2917636036872864,
  'p90': 1.3048157691955566},
 192: {'mse': 0, 'mae': 0, 'p10': 0, 'p50': 0, 'p90': 0},
 336: {'mse': 0, 'mae': 0, 'p10': 0, 'p50': 0, 'p90': 0},
 720: {'mse': 0, 'mae': 0, 'p10': 0, 'p50': 0, 'p90': 0}}

In [ ]:
# we load the pretrained model again and do full fine tuning on the target dataset

best_parameters = {'depth': 2, 'dim': 256, 'dim_head': 56, 'heads': 4, 'attn_dropout': 0.2, 'ff_mult': 4, 'ff_dropout': 0.1, 
                   'num_mem_tokens': 4, 'learning_rate': 0.0005}


model_config = {
    'num_variates': data_dict["train"].size(1),
    'lookback_len': window_size,
    'depth': best_parameters["depth"],
    'dim': best_parameters["dim"],
    'num_tokens_per_variate': 1,
    'pred_length': pred_length,
    'dim_head': best_parameters["dim_head"],
    'heads': best_parameters["heads"],
    'attn_dropout': best_parameters["attn_dropout"],
    'ff_mult': best_parameters["ff_mult"],
    'ff_dropout': best_parameters["ff_dropout"],
    'num_mem_tokens': best_parameters["num_mem_tokens"],
    'use_reversible_instance_norm': False,
    'reversible_instance_norm_affine': False,
    'flash_attn': True
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model_path = config.CONFIG_MODEL_LOCATION[normalization_strategie]
checkpoint = torch.load(model_path)
model = iTransformer(**model_config).to(device)
model.load_state_dict(checkpoint['model_state_dict'])


# defining all needed instances
optimizer = optim.Adam(model.parameters(), lr=best_parameters["learning_rate"])
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
writer = SummaryWriter(log_dir=config.CONFIG_LOGS_PATH[normalization_strategie])

# Fine-tuning with the full bavaria dataset and prediciton on the same test dataset

In [ ]:
# bavaria dataset
data_tensor = data_handling.load_bavaria_electricity()
data_dict, standadizer = data_handling.train_test_split_eu_elec(data_tensor, standardize=True)

# convert to datalaoder
dataloader_train, dataloader_validation, dataloader_test = data_handling.convert_data(data_dict, window_size, pred_length)

In [ ]:
# now we train on a small subset of the training data for multiple epochs and evaluate again

epoch = 10
training_functions.train_one_epoch(epoch, model, device, dataloader_train, dataloader_validation, optimizer, scheduler, writer)

metrics = training_functions.full_eval(model, dataloader_test)
metrics

for eval_metric, value in metrics[96].items():
    metrics[96][eval_metric] = value.item()

metrics_df = pd.DataFrame.from_dict(metrics[96], orient='index')
metrics_df.rename(columns={0: key}, inplace=True)

metrics_df.to_csv(f"{config.CONFIG_OUTPUT_PATH['iTransformer_baseline']}/metrics_{key}_epochs{epoch}_revin.csv")
